In [1]:
import json
import os
import logs_generator
import base_model

In [ ]:
os.remove('log.txt')
logs_generator.main()

In [6]:
logs = {}

with open('log.txt') as f:
    for row in f:
        try:
            log, nes = row.split(':')
            logs[log.strip()] = [ne.strip() for ne in nes.split(' , ')]
            if logs[log.strip()] == ['']:
                logs[log.strip()] = []
        except ValueError:
            logs[row.strip()] = []

In [7]:
len(logs)

989

In [9]:
with open('dicts_with_regex.json', 'r', encoding='utf-8') as f:
        dicts_with_regex = json.load(f)

In [10]:
found_nes = base_model.find_nes(logs, dicts_with_regex)

ИС найдены в 808 логах из 989


In [11]:
processed_logs = {}
for log_nes in found_nes:
    log = log_nes['log']
    nes = [ne['ne'] for ne in log_nes['nes']]
    processed_logs[log] = nes

In [12]:
def count_metrics(logs, processed_logs):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for log in processed_logs:
        for ne in processed_logs[log]:
            if ne in logs[log]:
                true_positives += 1
            else:
                false_positives += 1
        
    for log in logs:
        if log in processed_logs:
            for ne in logs[log]:
                if ne not in processed_logs[log]: 
                    false_negatives += 1
        else:
            false_negatives += len(logs[log])
    
    metrics = {}
    precision = true_positives / (true_positives + false_positives)
    metrics['precision'] = precision
    recall = true_positives / (true_positives + false_negatives)
    metrics['recall'] = recall
    metrics['f1'] = 2 * precision * recall / (precision + recall)
    return metrics

In [13]:
metrics = count_metrics(logs, processed_logs)
for metric in metrics:
    print(f'{metric} {round(metrics[metric], 2)}')

precision 0.81
recall 0.73
f1 0.77
